# Aggregate data on scores

Aggregate the synthetic person-level responses to find the mean scores.

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    survey = '../data/survey_data'
    synthetic_data = 'synthetic_data_raw.csv'
    aggregate_score = 'aggregate_scores.csv'


paths = Paths()

### Import raw data

In [3]:
data = pd.read_csv(os.path.join(paths.survey, paths.synthetic_data))
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_listen_lab,peer_talk_helpful_lab,peer_talk_if_lab,accept_peer_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab
0,4.0,2.0,NaN,3.0,2.0,1.0,2.0,1.0,2.0,5.0,...,Fully,Somewhat helpful,Very uncomfortable,Not at all,Year 10,Non-FSM,Non-SEN,NaN,No,School E
1,1.0,2.0,1.0,3.0,3.0,2.0,3.0,8.0,4.0,2.0,...,Mostly,Very helpful,Very uncomfortable,Slightly,Year 10,Non-FSM,Non-SEN,Ethnic minority,No,School D
2,2.0,3.0,4.0,1.0,NaN,1.0,1.0,1.0,5.0,4.0,...,Mostly,Very helpful,Very comfortable,Not at all,Year 10,Non-FSM,Non-SEN,White British,No,School E
3,2.0,5.0,5.0,2.0,2.0,2.0,1.0,3.0,1.0,2.0,...,Fully,Somewhat helpful,Uncomfortable,Mostly,Year 10,Non-FSM,Non-SEN,White British,No,School G
4,5.0,3.0,4.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,...,Slightly,Somewhat helpful,Uncomfortable,Not at all,Year 8,Non-FSM,Non-SEN,White British,Yes,School B


## Find mean score per school

Find mean score by school:
* Overall
* By year
* By gender (this is gender from survey, not council, as can see from GM data extract)
* By FSM
* By SEN

Hide results when count < 10.

In [4]:
# Find columns that provide a score
score_col = [col for col in data.columns if col.endswith('_score')]


def aggregate(calc, group_col):
    '''
    Aggregate the score columns of data, using provided group, either finding
    the number of non-NaN or the mean (ignoring NaN)
    Inputs:
    - calc: 'mean' or 'count'
    - group_col: list of columns to groupby
    Output:
    - result: dataframe with mean or count for each score + group as each row
    '''
    # Find the mean of each score, ignoring NaN
    if calc == 'mean':
        result = data[group_col + score_col].groupby(group_col).mean()
    # Count non-NaN, so know what was used in mean, as there are pupils
    # missing data for that score or characteristic (e.g. year group)
    elif calc == 'count':
        result = data[group_col + score_col].groupby(group_col).count()

    # Transform from wide to long
    result = pd.melt(result.reset_index(), id_vars=group_col)

    # Rename the value column to the use calculation
    result = result.rename(columns={'value': calc})

    return(result)


def mean_and_count(group_col):
    '''
    Uses the aggregate() function to find the mean and counts for each score
    column when grouped by the provided columns
    Inputs:
    - group_col: list of columns to groupby
    Outputs:
    - result: dataframe with mean and count for each score + group as each row
    '''
    # Find mean and count
    res_mean = aggregate('mean', group_col)
    res_count = aggregate('count', group_col)

    # Combine the results dataframe - using outer (all ID) rather than inner
    # (only matching ID) - although all columns should match
    result = pd.merge(left=res_mean, right=res_count, how='outer')

    # Hide results when n<10
    result.loc[result['count'] < 10, ['mean', 'count']] = np.nan

    return(result)

In [5]:
# Create the groups - school alone or combined with a filter
groups = [['school_lab']]
filters = ['year_group_lab', 'gender_lab', 'fsm_lab', 'sen_lab']
for group in filters:
    groups.append(['school_lab'] + [group])
groups

[['school_lab'],
 ['school_lab', 'year_group_lab'],
 ['school_lab', 'gender_lab'],
 ['school_lab', 'fsm_lab'],
 ['school_lab', 'sen_lab']]

In [6]:
# Find mean and count for each of the groups and add to a list
agg_results = []
for group in groups:
    agg_results.append(mean_and_count(group))

# Combine the seperate aggregation results into one dataframe
agg = pd.concat(agg_results)
agg

,school_lab,variable,mean,count,year_group_lab,gender_lab,fsm_lab,sen_lab
0,School A,birth_you_age_score,7.861111,108.0,NaN,NaN,NaN,NaN
1,School B,birth_you_age_score,7.900000,110.0,NaN,NaN,NaN,NaN
2,School C,birth_you_age_score,8.360465,86.0,NaN,NaN,NaN,NaN
3,School D,birth_you_age_score,8.152174,92.0,NaN,NaN,NaN,NaN
4,School E,birth_you_age_score,7.652174,92.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
424,School E,bully_score,8.000000,30.0,NaN,NaN,NaN,SEN
425,School F,bully_score,7.051282,39.0,NaN,NaN,NaN,Non-SEN
426,School F,bully_score,7.781250,32.0,NaN,NaN,NaN,SEN
427,School G,bully_score,7.744186,43.0,NaN,NaN,NaN,Non-SEN


In [7]:
# Set NaN for the filter labs as All
agg[filters] = agg[filters].fillna('All')
agg

,school_lab,variable,mean,count,year_group_lab,gender_lab,fsm_lab,sen_lab
0,School A,birth_you_age_score,7.861111,108.0,All,All,All,All
1,School B,birth_you_age_score,7.900000,110.0,All,All,All,All
2,School C,birth_you_age_score,8.360465,86.0,All,All,All,All
3,School D,birth_you_age_score,8.152174,92.0,All,All,All,All
4,School E,birth_you_age_score,7.652174,92.0,All,All,All,All
...,...,...,...,...,...,...,...,...
424,School E,bully_score,8.000000,30.0,All,All,All,SEN
425,School F,bully_score,7.051282,39.0,All,All,All,Non-SEN
426,School F,bully_score,7.781250,32.0,All,All,All,SEN
427,School G,bully_score,7.744186,43.0,All,All,All,Non-SEN


<mark>Groupby only uses groups present - see example below where no SEN- is this an issue?</mark>

In [8]:
agg[agg['school_lab'] == 'School B'].sen_lab.value_counts(dropna=False)

sen_lab
All        363
Non-SEN     33
Name: count, dtype: int64

## Find overall counts for the grouping columns

We have previously found counts who have a score (with counts excluding pupils NaN for a given score).

This is to calculate the overall counts:
* Per school
* Per year group (so excluding NaN for year group)

And so on for the other grouping columns

In [9]:
# For each of the grouping columns used above, find the size
size_list = []
for group_col in groups:
    size_list.append(data.groupby(group_col).size().reset_index(name='count'))

# Combine the size results into a single dataframe
size = pd.concat(size_list)

# Replace NaN in the group columns with 'All'
size[filters] = size[filters].fillna('All')

# Set the variable as overall count
size['variable'] = 'overall_count'

# Preview dataframe
size

,school_lab,count,year_group_lab,gender_lab,fsm_lab,sen_lab,variable
0,School A,116,All,All,All,All,overall_count
1,School B,119,All,All,All,All,overall_count
2,School C,97,All,All,All,All,overall_count
3,School D,96,All,All,All,All,overall_count
4,School E,97,All,All,All,All,overall_count
...,...,...,...,...,...,...,...
8,School E,43,All,All,All,SEN,overall_count
9,School F,45,All,All,All,Non-SEN,overall_count
10,School F,41,All,All,All,SEN,overall_count
11,School G,47,All,All,All,Non-SEN,overall_count


## Combine and save results

In [10]:
agg_size = pd.concat([agg, size])

In [11]:
agg_size.to_csv(os.path.join(paths.survey, paths.aggregate_score), index=False)